## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [4]:
use_hydroelastic = False # this takes forever

def make_builder():
    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    return builder, plant, station

# diagram = builder.Build()


In [5]:
from scripts.grasp_broom import plan_path, build_temp_plant, combine_trajectory

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


In [ ]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)
# meshcat.SetObject("gripper", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("gripper", gripper_frame.GetPoseInWorld(context))


In [ ]:
# path = plan_path(X_WG1, X_WG2)

In [6]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WGoal = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.7, 0.5])

AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
AddMeshcatTriad(meshcat, 'b', X_PT=X_WGoal)

traj = plan_path(X_WStart, X_WGoal)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()


7
(7, 10)
Collision optimization failed
time0.0: [-0.91985197  0.51987411 -0.85267042 -1.30204739  0.00558833  1.73384652
  1.37104912]
time0.08704913697384586: [-0.87173397  0.50636951 -0.82880341 -1.29884608  0.00307587  1.72785707
  1.29887734]
time0.17409827394769173: [-7.54326072e-01  4.69621834e-01 -7.74717276e-01 -1.29455990e+00
 -1.65182933e-03  1.71625725e+00  1.13014229e+00]
time0.2611474109215376: [-0.60789025  0.41529576 -0.71654755 -1.29711825 -0.00438457  1.70853888
  0.93628558]
time0.34819654789538346: [-4.58999437e-01  3.50405265e-01 -6.68515271e-01 -1.31033986e+00
 -1.28693194e-03  1.70888153e+00  7.68814555e-01]
time0.4352456848692293: [-0.31010862  0.28434155 -0.62385007 -1.33080108  0.01081616  1.71210545
  0.64411566]
time0.5222948218430752: [-0.16121792  0.22662488 -0.57432302 -1.35420602  0.03445189  1.71239461
  0.57416687]
time0.609343958816921: [-0.01232946  0.18487871 -0.52309668 -1.37399058  0.06239407  1.70924755
  0.55694655]
time0.6963930957907669: [ 0.1

In [ ]:
from scripts.broom_utils import get_broom_pregrip, get_broom_grip,compute_broom_grasp_angle
plant_context = plant.CreateDefaultContext()
base = plant.GetBodyByName("base", plant.GetModelInstanceByName('iiwa'))
body = plant.GetBodyByName("handle_link")
X_WBroom = plant.EvalBodyPoseInWorld(plant_context, body)
X_WBase = plant.EvalBodyPoseInWorld(plant_context, base)
ang = compute_broom_grasp_angle(X_WBroom, X_WBase.translation())
ang = 0
X_WPregrip = get_broom_pregrip(X_WBroom, ang)
X_WGrip = get_broom_grip(X_WBroom, ang)
AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
AddMeshcatTriad(meshcat, 'handle', X_PT=X_WPregrip)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrip)

builder, plant, station = make_builder()

traj = plan_path(X_WStart, X_WPregrip)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
X_WPregrip


[6.00000000e-01 2.12072520e-07 5.23204958e-01]
1.9819510504590978


TypeError: SetTransform(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.geometry.Meshcat, path: str, X_ParentPath: pydrake.math.RigidTransform, time_in_recording: Optional[float] = None) -> None
    2. (self: pydrake.geometry.Meshcat, path: str, matrix: numpy.ndarray[numpy.float64[4, 4], flags.f_contiguous]) -> None

Invoked with: <pydrake.geometry.Meshcat object at 0x71033c0785f0>, 'handle', None

In [53]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
# simulator.AdvanceTo(0.1)
meshcat.StopRecording()
meshcat.PublishRecording()